# Task
Our task is simple, recognize handwritten digits. We will use MNIST dataset for this tutorial.


# Import necessary library
In this tutorial, we are going to use pytorch, the cutting-edge deep learning framework to complete our task.

In [3]:
import torch
import torchvision

In [4]:
## Create dataloader, in PyTorch, we feed the trainer data with use of dataloader
## We create dataloader with dataset from torchvision, 
## and we dont have to download it seperately, all automatically done

# Define batch size, batch size is how much data you feed for training in one iteration
batch_size_train = 64 # We use a small batch size here for training
batch_size_test = 1024 #

# define how image transformed
image_transform = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])
#image datasets
train_dataset = torchvision.datasets.MNIST('dataset/', 
                                           train=True, 
                                           download=True,
                                           transform=image_transform)
test_dataset = torchvision.datasets.MNIST('dataset/', 
                                          train=False, 
                                          download=True,
                                          transform=image_transform)
#data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size_train, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size_test, 
                                          shuffle=True)

In [5]:
## Now we can start to build our CNN model
## We first import the pytorch nn module and optimizer
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
## Then define the model class
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #input channel 1, output channel 10
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1)
        #input channel 10, output channel 20
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        #dropout layer
        self.conv2_drop = nn.Dropout2d()
        #fully connected layer
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        print("Input",x.shape)
        x = self.conv1(x)
        print("Conv1",x.shape)
        x = F.max_pool2d(x, 2)
        print("pool1",x.shape)
        x = F.relu(x)
        x = self.conv2(x)
        print("Conv2",x.shape)
        print(x.shape)

        x = F.max_pool2d(x, 2)
        print("pool2",x.shape)
        

        x = F.relu(x)

        x = x.view(-1, 320)
        print(x.shape)

        x = self.fc1(x)
        print("FC1",x.shape)

        x = F.relu(x)

        x = F.dropout(x)
        x = self.fc2(x)
        print("FC2",x.shape)


        return F.log_softmax(x)

In [ ]:
import numpy as np
model = CNN()
x=torch.randint(low=0, high=255, size=[1,28,28], dtype=torch.float32)
model(x)

Input torch.Size([1, 28, 28])
Conv1 torch.Size([10, 24, 24])
pool1 torch.Size([10, 12, 12])
Conv2 torch.Size([20, 8, 8])
torch.Size([20, 8, 8])
pool2 torch.Size([20, 4, 4])
torch.Size([1, 320])
FC1 torch.Size([1, 50])
FC2 torch.Size([1, 10])


/tmp/ipykernel_28106/317683752.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


tensor([[-1.2628e+01, -8.3452e+00, -4.6015e-03, -2.4013e+01, -1.1289e+01,
         -5.6501e+00, -7.3985e+00, -8.7154e+00, -1.1177e+01, -1.0404e+01]],
       grad_fn=<LogSoftmaxBackward0>)

In [6]:
## create model and optimizer
learning_rate = 0.01
momentum = 0.5
device = "cpu"
model = CNN().to(device) #using cpu here
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [7]:
from tqdm import tqdm_notebook as tqdm
##define train function
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        counter += 1
        tk0.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))
##define test function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
num_epoch = 3
for epoch in range(1, num_epoch + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

/tmp/ipykernel_4881/1895196525.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tk0 = tqdm(train_loader, total=int(len(train_loader)))


  0%|          | 0/938 [00:00<?, ?it/s]

Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 

/tmp/ipykernel_4881/317683752.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 

  0%|          | 0/938 [00:00<?, ?it/s]

Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 

  0%|          | 0/938 [00:00<?, ?it/s]

Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 4])
torch.Size([64, 320])
FC1 torch.Size([64, 50])
FC2 torch.Size([64, 10])
Input torch.Size([64, 1, 28, 28])
Conv1 torch.Size([64, 10, 24, 24])
pool1 torch.Size([64, 10, 12, 12])
Conv2 torch.Size([64, 20, 8, 8])
torch.Size([64, 20, 8, 8])
pool2 torch.Size([64, 20, 4, 

In [8]:
# from torchsummary import summary
# summary(model, (1, 28, 28))

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)